In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as mpatches
import lzma

pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 1000
#plt.style.use('ggplot')
plt.style.use('seaborn-paper')
mpl.rcParams['axes.linewidth'] = 0.7

In [ ]:
file = "latest_results.json.xz"

data = None
if file.endswith('.xz'): file = lzma.open(file, 'rt')
data = pd.read_json(file)
data = data.sort_values(['file', 'method'])

In [ ]:
data = data[data.method != 'reforest']

In [ ]:
# Adding a column for the database
data['db'] = -1 + \
    (0+1)*data.file.str.endswith(')') + \
    (1+1)*data.file.str.contains('/Prover9--') + \
    (2+1)*data.file.str.contains('/Vampire---4.0') + \
    (3+1)*data.file.str.contains('/leanCoP--') + \
    (4+1)*data.file.str.contains('/QF_UF/')
    
data['status'] = data.status.fillna("timeout_" + data.phase)

data['is_smaller'] = (data.status=='ok') & (data.ehs_lkinf < data.hs_lkinf)
data['is_larger'] = (data.status=='ok') & (data.ehs_lkinf >= data.hs_lkinf)
data['is_trivial'] = data.status.str.endswith('termset_trivial')
data['is_uncompressible'] = data.status.str.endswith('uncompressible')
data['is_timeout'] = data.status.str.startswith('timeout')
data['is_error'] = ~data.is_smaller & ~data.is_larger & ~data.is_trivial & ~data.is_uncompressible & ~data.is_timeout

data['tstp_cat'] = data.file.str.extract('/([A-Z]{3,3})/', expand=False)
data['is_tstp'] = data.tstp_cat.notnull()

data['compression_ratio'] = data.termset/data.grammar_size
data['inv_compression_ratio'] = data.grammar_size/data.termset

data['lk_cmp_ratio'] = data.ehs_lkinf/data.hs_lkinf

data['has_sol'] = data.time_beausol>=0

In [ ]:
# Other exceptions (aka bugs to hunt)

data_other_exp = data[(data.status == 'cutintro_other_exception')]
data_other_exp[['method', 'file', 'status']].sort_index()

In [ ]:
methods = sorted(set(data.method.dropna()))

dbs = {
    0: 'Proof examples',
    1: 'Prover9',
    2: 'Vampire',
    3: 'LeanCoP',
    4: 'VeriT',
}

dbs = { i: dbs[i] for i in dbs.keys() if i in data.db.values }

In [ ]:
#print("=== Number of tests per database and method:\n")

#for dbi, dbname in dbs.items():
#    for method in methods:
#        print("{0} - {1}: {2}".format(dbname, method,
#                                     data[(data.method==method) & (data.db==dbi)].shape[0]))
#    print()

In [ ]:
states = set(data.status.values)
status_colors = {}
for s in states:
    if s == 'ok':
        c = 'green'
    elif s == 'cutintro_uncompressible':
        c = 'yellow'
    elif s == 'cutintro_termset_trivial':
        c = 'gold'
    elif 'timeout' in s:
        c = plt.cm.Blues(hash(s)/2**65+0.5)
    elif s.endswith('exception'):
        c = plt.cm.Reds(hash(s)/2**65+0.5)
    else:
        c = plt.cm.Greys(hash(s)/2**65+0.5)
    status_colors[s] = c

def plot_status(d, legend=True, **kwargs):
    plt.axis('equal')
    d.plot(kind='pie', autopct='%1.1f%%',
                        colors=[status_colors[s] for s in d.index], **kwargs) \
        .set_ylabel('')
    if legend: plt.legend(d.index, loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
plot_status(data.groupby('status').size(), figsize=(14,12), title='Return status: full set')

In [ ]:
xsize, ysize = len(dbs.items()), len(methods)
plt.figure(1, figsize=(3*xsize,4*ysize))
plt.suptitle('Status per method and db', fontsize=25)
for j, (dbi, db) in enumerate(dbs.items()):
    for i, method in enumerate(methods):
        plt.subplot(ysize, xsize, i*xsize + j + 1)
        subdata = data[(data.db==dbi) & (data.method==method)].groupby('status').size()
        plot_status(subdata, title='{0}\n{1}'.format(db,method), labels=None, legend=False)

In [ ]:
timecols = ['time_grammar', 'time_minsol', 'time_prcons', 'time_cleanproof']
#plt.axis('equal')
#data[timecols].mean().plot(kind='pie', figsize=(12,12), autopct='%1.1f%%',
#                           title='Time: full set', legend=True).set_ylabel('')

In [ ]:
#xsize, ysize = len(dbs.items()), len(methods)
#plt.figure(1, figsize=(3*xsize,4*ysize))
#plt.suptitle('Time consumption per method and db', fontsize=25)
#for i, method in enumerate(methods):
#    for j, (dbi, db) in enumerate(dbs.items()):
#        plt.subplot(ysize, xsize, i*xsize + j + 1)
#        plt.axis('equal')
#        data[(data.db==dbi) & (data.method==method)][timecols].mean() \
#            .plot(kind='pie', autopct='%1.1f%%', labels=None,
#                  title='{0}\n{1}'.format(db,method)).set_ylabel('')
#plt.legend(timecols, loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
#xsize, ysize = len(dbs.items()), len(methods)
#plt.figure(1, figsize=(3*xsize,4*ysize))
#plt.suptitle('Time consumption per method and db, state==ok', fontsize=25)
#for i, method in enumerate(methods):
#    for j, (dbi, db) in enumerate(dbs.items()):
#        plt.subplot(ysize, xsize, i*xsize + j + 1)
#        plt.axis('equal')
#        ax = data[(data.db==dbi) & (data.method==method) & (data.status=='ok')][timecols].mean() \
#            .plot(kind='pie', autopct='%1.1f%%', labels=None,
#                  title='{0}\n{1}'.format(db,method))
#        ax.set_ylabel('')
#plt.legend(timecols, loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
data['status'].value_counts()

In [ ]:
# Different grammar sizes between different methods: one cut with one quantifier
data_dtable = data[(data.method == '1_dtable') & data.grammar_size]
data_maxsat = data[(data.method == '1_maxsat') & data.grammar_size]

merged = pd.merge(data_dtable, data_maxsat, on=['file'], suffixes=('_dtable', '_maxsat'))
merged[merged.grammar_size_dtable != merged.grammar_size_maxsat] \
    [['file', 'grammar_size_dtable', 'grammar_size_maxsat']]

In [ ]:
# Different grammar sizes between different methods: one cut with multiple quantifiers
data_dtable = data[(data.method == 'many_dtable') & data.grammar_size]
data_maxsat = data[(data.method == '2_maxsat') & data.grammar_size]

merged = pd.merge(data_dtable, data_maxsat, on=['file'], suffixes=('_dtable', '_maxsat'))
merged[merged.grammar_size_dtable != merged.grammar_size_maxsat] \
    [['file', 'grammar_size_dtable', 'grammar_size_maxsat']]

In [ ]:
# Better compression with 2x2 quantifiers than 1x2 quantifiers or 2x1 quantifiers
data_1x2 = data[(data.method == '2_maxsat') & data.grammar_size]
data_2x1 = data[(data.method == '1_1_maxsat') & data.grammar_size]
data_2x2 = data[(data.method == '2_2_maxsat') & data.grammar_size]

merged = data_1x2.merge(data_2x1, on='file', suffixes=('_1x2', '_2x1')) \
    .merge(data_2x2, on='file')
merged[(merged.grammar_size < merged.grammar_size_1x2) & (merged.grammar_size < merged.grammar_size_2x1)] \
    [['file', 'grammar_size', 'grammar_size_2x1', 'grammar_size_1x2']]

In [ ]:
data['is_ok'] = data['status']=='ok'
summary = (data[(data.termset<65)&data.is_tstp&(data.method=='many_dtable_ss')].groupby('termset').sum() \
    [['is_trivial', 'is_uncompressible', 'is_ok', 'is_timeout']] * 1)
summary.columns = ['trivial', 'uncompressible', 'lemma generated', 'timeout']
summary.plot(kind='area', figsize=(5.5,4), ylim=(0,5500),
          title='Status by termset size for the delta-table with many variables',
          color=['gold', 'yellow', 'lime', 'lightblue', 'tomato'])
plt.ylabel("Number of proofs")
plt.xlabel("Termset size")
plt.savefig('status_by_termset.pdf')

In [ ]:
# Proofs with good compression of EHS/HS
data[(data.ehs_lkinf/data.hs_lkinf < 0.2)].sort_values('lk_cmp_ratio') \
    [['file', 'method', 'ehs_lkinf', 'hs_lkinf', 'lkinf_input', 'beausol']]

In [ ]:
# Proofs with blowup
data[data.ehs_lkinf > 5*data.hs_lkinf] \
    [['file', 'method', 'ehs_lkinf', 'hs_lkinf', 'lkinf_input']]

In [ ]:
# Cactus plot for grammar finding methods

plt.figure(figsize=(5.5,3))

tstp_dataset = \
    (data.grammar_size>0)& \
    data.beausol& \
    data.is_tstp
for marker, method in zip("*sov^<>1", sorted(set(data[tstp_dataset].method),
                     key = lambda m: len(data[tstp_dataset&(data.method==m)&(data.time_grammar<5000)]))):
    plt.plot(
        data[(data.method==method)&tstp_dataset].
            time_grammar.sort_values().values / 1000,
        label=method,
        marker=marker, markevery=0.1)

virtual_best = data[tstp_dataset].groupby('file').time_grammar.min()
plt.plot(virtual_best.sort_values().values / 1000, label='virtual best', color='orange',
         marker='D', markevery=0.1)

plt.xlabel('Number of grammars that lead to a non-trivial lemma')
plt.ylabel('CPU runtime (seconds)')
plt.ylim(0,5)
plt.xlim(0,12000)
plt.legend(loc='upper right', framealpha=0.7)
plt.savefig('grammar_finding_cactus.pdf')

In [ ]:
data[(data.beausol_scomp < 100) &
     (data.beausol_lcomp > 3) &
     (data.beausol_scomp > 1) &
     (data.time_total > 30000) &
     data.is_tstp &
     ~data.file.str.contains('SY')] \
    .sort_values('beausol_scomp')[['file', 'method', 'beausol', 'time_total']]

In [ ]:
plt.figure(figsize=(5.5,3))
tstp_success = data[data.is_tstp&data.beausol]
(tstp_success.minsol_scomp/tstp_success.cansol_scomp).plot.kde(label='minimized', marker='s', markevery=0.1)
(tstp_success.beausol_scomp/tstp_success.cansol_scomp).plot.kde(label='beautified', marker='o', markevery=0.1)
plt.legend()
plt.xlim(0,1)
plt.ylabel('Estimated density')
plt.xlabel('Ratio of symbolic complexity compared to canonical solution')
plt.savefig('improvement_comp_density.pdf')

In [ ]:
# Added productions during beautification
tstp_success = data[data.is_tstp&data.beausol]
(tstp_success.beaugrammar_size-tstp_success.grammar_size).describe(percentiles = [.25, .5, .75, .80, .85])

In [ ]:
plt.figure(figsize=(14,8))
tstp_ok = data[data.is_tstp&(data.status=='ok')&data.ehs_lkinf]
(np.log(tstp_ok.ehs_lkinf/tstp_ok.hs_lkinf)/np.log(10)).plot.hist(label='proof of EHS', bins=100)
plt.legend()
plt.ylabel('Number of proofs')
plt.xlabel('Logarithmic ratio of number of inferences of proof of EHS compared to proof of HS')

In [ ]:
data[data.is_tstp&data.beausol&(data.method!='reforest')].plot.scatter('termset', 'compression_ratio',
                          alpha=0.2, figsize=(14,8))
plt.xlim(0,500)

In [ ]:
plt.figure(figsize=(14,8))
data[data.is_tstp&data.beausol&data.is_tstp&(data.termset_trivial==False)].inv_compression_ratio.plot.hist(bins=100)
plt.xlabel('Compression ratio (size of term set / size of grammar)')
plt.xlim(0, 1)

In [ ]:
# Number of proofs in TSTP library
num_tstp = len(set(data[data.is_tstp].file)); num_tstp

In [ ]:
# Number of proofs we can parse
num_tstp_parsed = len(set(data[data.is_tstp&data.termset].file)); num_tstp_parsed

In [ ]:
# Number of trivial termsets
num_trivial = len(set(data[data.is_tstp&data.termset_trivial].file)); num_trivial

In [ ]:
# Number of non-trivial termsets
num_nontrivial = len(set(data[data.is_tstp&(data.termset_trivial==False)].file)); num_nontrivial

In [ ]:
# Number of grammars generated
num_grammar_gen = len(set(data[data.is_tstp&data.grammar_size].file)); num_grammar_gen

In [ ]:
# Number of lemmas generated
num_lemmas_gen = len(set(data[data.is_tstp&data.beausol].file)); num_lemmas_gen

In [ ]:
# Solutions for trivial termsets
data[data.is_tstp &
     data.termset_trivial &
     (data.grammar_size>0) &
     (data.beausol_scomp<10) &
     (data.beausol_scomp>1)
    ][['file', 'beausol']]

In [ ]:
# Number of proofs with trivial termsets where we found a grammar
len(set(data[data.is_tstp&data.termset_trivial&(data.grammar_size>0)].file))

In [ ]:
# Size of canonical solution vs grammar size
tstp_success = data[data.is_tstp&data.beausol]
tstp_success.plot.scatter('cansol_lcomp', 'grammar_size', figsize=(14,8), alpha=0.2)

In [ ]:
data[data.status=='cutintro_noncovering_grammar'][['file','method']]

In [ ]:
(data[data.time_total>0].time_total/1000).plot.hist(xlim=(0,60), bins=100, figsize=(5.5,3))
plt.xlabel('Seconds')
plt.ylabel('Number of proofs')
plt.savefig('timedist.pdf')

In [ ]:
data[data.lk_cmp_ratio<10].lk_cmp_ratio.plot.hist(bins=100)

In [ ]:
data[data.lk_cmp_ratio<1].plot.scatter('lk_cmp_ratio', 'inv_compression_ratio', alpha=0.1)

In [ ]:
data[data.lk_cmp_ratio<1].plot.scatter('termset', 'lk_cmp_ratio')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(5.5,3))
data[data.is_tstp&data.has_sol].plot.scatter('termset', 'inv_compression_ratio', ax=axes[0],
                                             c='black', alpha=0.1, ylim=(0,1), xlim=(0,700), s=3)
data[data.is_tstp].plot.scatter('termset', 'inv_compression_ratio', ax=axes[1],
                                c='black', alpha=0.1, ylim=(0,1), xlim=(0,700), s=3)
axes[0].set_ylabel('Compression ratio')
axes[1].set_ylabel('')
for i in range(2): axes[i].set_xlabel('Termset size')
axes[0].set_title('Lemma was generated')
axes[1].set_title('All proofs')
for ax in axes: ax.set_rasterized(True)
plt.savefig('ratio_scatter.pdf', rasterized=True, dpi=300)

In [ ]:
data.is_tstp.sum()